In [ ]:
# load a bunch of masks

In [ ]:
import json

import numpy as np
import matplotlib.pyplot as plt
from pycocotools.mask import decode


In [ ]:
# load labels
labels = json.load(open('/root/data/blender_v2/training/train_low_rez/labels.json'))

In [ ]:
masks = decode(labels[0]['masks'])

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(masks[...,0])

In [ ]:
plt.figure(figsize=(15, 15))
plt.imshow(np.sum(masks,axis=2))# [200:, :500])

# Create a dataset of masks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.cm as cm

import glob
import os

In [ ]:
# load some crops 
crop_folder = '/root/data/blender_v3/crop/'

In [ ]:
crops = [os.path.join(crop_folder, f) for f in os.listdir(crop_folder) if f.endswith('.png') and f[:4] == 'left']
print("{} crops found".format(len(crops)))

In [ ]:
if not os.path.isdir('/root/data/mask_dataset'):
    os.makedirs('/root/data/mask_dataset/train/occluded')
    os.makedirs('/root/data/mask_dataset/train/full')

In [ ]:
train_size = 5000

In [ ]:
crops[0]

In [ ]:
# # create the full crops
# x_full = np.zeros((train_size, 28, 28, 1))
# for (i, crop) in enumerate(crops):
#     if i % 100 ==0 : print('{}/{}'.format(i, train_size))
#     mask = np.expand_dims(np.asarray(Image.open(crop).resize((28, 28)))[...,3], axis=2)
#     x_full[i,...] = mask
# plt.imshow(np.squeeze(x_full[np.random.randint(train_size),...]))

In [ ]:
# np.save('/root/data/mask_dataset/train/full/train_full', x_full)

In [ ]:
# create occluded dataset
for i in range(train_size):
    fish1 = np.asarray(Image.open(np.random.choice(crops)).resize((28, 28)))[...,3]
    fish1.flags.writeable = True
    fish1[fish1 > 0] = 1
    fish2 = np.asarray(Image.open(np.random.choice(crops)).resize((28, 28)))[...,3]
    fish2.flags.writeable = True
    fish2[fish2 > 0] = 1
    overlap = fish1 + fish2
    break

In [ ]:
plt.imshow(overlap)

# Keras convnet

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import os
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
crop_folder = '/root/data/blender_v3/crop/'
crops = [os.path.join(crop_folder, f) for f in os.listdir(crop_folder) if f.endswith('.png') and f[:4] == 'left']

In [ ]:
batch_size = 128
num_classes = 2
epochs = 20

In [ ]:
classes = ['full', 'occluded']

In [ ]:
# input image dimensions
input_shape = (28, 28, 1)
img_rows, img_cols = 28, 28

In [ ]:
# # the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# print(x_train.shape)
# plt.imshow(x_train[0])
# plt.colorbar()
# plt.show()
# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)

# x_train = x_train.astype('float32')
# plt.imshow(x_train[0,...,0])
# plt.colorbar()
# plt.show()
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
# plt.imshow(x_train[0,...,0])
# plt.colorbar()
# plt.show()
# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# # convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# fish occlusion function
def occlude(path1, path2):
    """take one fish and occlude it with the other"""
    # create the binary masks
    fish1 = np.asarray(Image.open(path1).resize((28, 28)))[...,3]
    fish1.flags.writeable = True
    fish1[fish1 > 0] = 1
    fish2 = np.asarray(Image.open(path2).resize((28, 28)))[...,3]
    fish2.flags.writeable = True
    fish2[fish2 > 0] = 1
    
    # randomly sample upper left
#     upper_x = np.random.randint(14)
#     upper_y = np.random.randint(14)
    
    coin = np.random.rand()
    if coin < 0.25:
        overlap = fish1[]
    fish1[overlap==2] =0
    return fish1

In [ ]:
overlap = occlude(np.random.choice(crops), np.random.choice(crops))
plt.imshow(overlap)
plt.show()

In [ ]:
plt.imshow(np.asarray(Image.open(np.random.choice(crops)).resize((28, 28)))[...,3])

In [ ]:
def get_1_hot(y, classes):
    return np.squeeze(keras.utils.to_categorical(y, len(classes)))

In [ ]:
# define augmentation
seq = iaa.Sequential([
    iaa.CoarseDropout(0.5, size_percent=0.001)
], random_order=True)

In [ ]:
def generator(crops, classes, batch_size):
    i = 0
    while True:
        x_batch = np.empty((batch_size, 28, 28, 1))
        y_batch = np.empty((batch_size, len(classes)))
        for index in range(batch_size):
            coin = np.random.rand()
            if coin > 0.5:
                # add a non occluded fish
                fish = np.random.choice(crops)
                x_batch[index, ...] = np.expand_dims(np.asarray(Image.open(fish).resize((28, 28)))[...,3], axis=2)
                y_batch[index, ...] = get_1_hot(0, classes)
            else:
                # add an occluded fish
                coin2 = np.random.rand()
                if coin2 > 0.5:
                    # either occlude with squares
                    fish = np.random.choice(crops)
                    x_batch[index, ...] = seq.augment_image(np.expand_dims(np.asarray(Image.open(fish).resize((28, 28)))[...,3], axis=2))
                else:
                    # or with other fish
                    fish1 = np.random.choice(crops)
                    fish2 = np.random.choice(crops)
                    x_batch[index, ...] = np.expand_dims(occlude(fish1, fish2), axis=2)
                y_batch[index, ...] = get_1_hot(1, classes)
        yield x_batch, y_batch

In [ ]:
# fish = np.random.choice(crops)
# out = seq.augment_image(np.asarray(Image.open(fish).resize((28, 28)))[...,3])
# plt.imshow(out)

In [ ]:
# g = generator(crops, classes, 8)

In [ ]:
# xb, yb = g.next()

In [ ]:
# yb

In [ ]:
# import matplotlib.pyplot as plt

In [ ]:
# xb.shape

In [ ]:
# plt.imshow(xb[0, ...,0])

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
train_generator = generator(crops, classes, batch_size)
val_generator = generator(crops, classes, batch_size)

In [ ]:
model.fit_generator(generator = train_generator, 
                    steps_per_epoch = len(crops)//batch_size,
                    epochs = 20,
                    verbose = 1,
                    validation_data = val_generator,
                    validation_steps = len(crops) // (10*batch_size)
                   )

# verify

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fish = np.random.choice(crops)
x = np.expand_dims(np.asarray(Image.open(fish).resize((28, 28)))[...,3], axis=2)

In [ ]:
model.predict(np.expand_dims(x, axis=0))

In [ ]:
occ = seq.augment_image(np.expand_dims(np.asarray(Image.open(fish).resize((28, 28)))[...,3], axis=2))

In [ ]:
plt.imshow(np.squeeze(occ))

In [ ]:
model.predict(np.expand_dims(occ, axis=0))

In [ ]:
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])